In [11]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# =======================
# 1. 读入数据
# =======================
jap = pd.read_csv("jap_vlaue_tax.csv", sep=",")
mex = pd.read_csv("mex_value_tax.csv", sep=",", header=None)

# =======================
# 2. 统一列名
# =======================
mex.columns = ["Country", "Year", "Month", "value", "tariff", "cov19"]
jap = jap.rename(columns={
    "General Customs Value": "value",
    "Real Tariff": "tariff"
})

# =======================
# 3. 只保留日本数据
# =======================
df = jap.copy()

# 添加 ln(1 + tariff)
df["ln_tariff"] = np.log(1 + df["tariff"])

# =======================
# 4. 生成滞后项 ln(value)
# =======================
df["ln_value"] = np.log(df["value"])
df["ln_value_lag"] = df.groupby("Country")["ln_value"].shift(1)

# 删除滞后后产生的 NaN
df = df.dropna()

# =======================
# 5. 回归模型
# y = α + ρ*y_{t-1} + β ln(1+tariff) + γ cov19
# =======================
model = smf.ols("ln_value ~ ln_value_lag + ln_tariff + cov19", data=df).fit()
print(model.summary())

# 拿到参数
alpha = model.params["Intercept"]
rho = model.params["ln_value_lag"]
beta = model.params["ln_tariff"]
gamma = model.params["cov19"]

print("\n参数：", alpha, rho, beta, gamma)

# =======================
# 6. 预测 2025年1月的 ln(value)
# =======================

# 取 2024年12月 作为滞后项
last = df[df["Year"]==2024].sort_values("Month").iloc[-1]
ln_value_last = last["ln_value"]

# 取 2025.01 的关税（若没有你需要自己填写或假设不变）
tariff_2025_01 = last["tariff"]   # 假设不变
ln_tariff_2025_01 = np.log(1 + tariff_2025_01)

# 计算 ln(value_2025_01)
ln_val_2025_01 = alpha + rho * ln_value_last + beta * ln_tariff_2025_01

# 转回 trade total 值
val_2025_01 = np.exp(ln_val_2025_01)

print("\n预测 2025年1月 日本总出口额（value）为：", val_2025_01)


                            OLS Regression Results                            
Dep. Variable:               ln_value   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     16.58
Date:                Sun, 23 Nov 2025   Prob (F-statistic):           8.53e-08
Time:                        20:14:06   Log-Likelihood:                 12.327
No. Observations:                  59   AIC:                            -16.65
Df Residuals:                      55   BIC:                            -8.344
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       12.5451      2.459      5.102   

In [25]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# =======================
# 1. 读入数据
# =======================
jap = pd.read_csv("jap_vlaue_tax.csv", sep=",")
mex = pd.read_csv("mex_value_tax.csv", sep=",", header=None)

# =======================
# 2. 统一列名
# =======================
mex.columns = ["Country", "Year", "Month", "value", "tariff", "cov19"]
jap = jap.rename(columns={
    "General Customs Value": "value",
    "Real Tariff": "tariff"
})

# =======================
# 3. 只保留日本数据
# =======================
df = jap.copy()

#只保留2022以后
# df = df[df["Year"] >= 2022]

# 添加 ln(1 + tariff)
df["ln_tariff"] = np.log(1 + df["tariff"])

# =======================
# 4. 生成滞后项 ln(value)
# =======================
df["ln_value"] = np.log(df["value"])
df["ln_value_lag"] = df.groupby("Country")["ln_value"].shift(1)

# 删除滞后后产生的 NaN
df = df.dropna()

# =======================
# 5. 回归模型
# y = α + ρ*y_{t-1} + β ln(1+tariff) + γ cov19
# =======================
model = smf.ols("ln_value ~ ln_value_lag + ln_tariff + cov19", data=df).fit()
print(model.summary())

# 拿到参数
alpha = model.params["Intercept"]
rho = model.params["ln_value_lag"]
beta = model.params["ln_tariff"]

# print("\n参数：", alpha, rho, beta, gamma)

# =======================
# 6. 预测 2025年1月的 ln(value)
# =======================

# 取 2024年12月 作为滞后项
last = df[df["Year"]==2024].sort_values("Month").iloc[-1]
ln_value_last = last["ln_value"]

# 取 2025.01 的关税（若没有你需要自己填写或假设不变）
tariff_2025_01 = last["tariff"]   # 假设不变
ln_tariff_2025_01 = np.log(1 + tariff_2025_01)

# 计算 ln(value_2025_01)
ln_val_2025_01 = alpha + rho * ln_value_last + beta * ln_tariff_2025_01

# 转回 trade total 值
val_2025_01 = np.exp(ln_val_2025_01)

print("\n预测 2025年1月 日本总出口额（value）为：", val_2025_01)


                            OLS Regression Results                            
Dep. Variable:               ln_value   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     16.58
Date:                Sun, 23 Nov 2025   Prob (F-statistic):           8.53e-08
Time:                        20:32:55   Log-Likelihood:                 12.327
No. Observations:                  59   AIC:                            -16.65
Df Residuals:                      55   BIC:                            -8.344
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       12.5451      2.459      5.102   

用于建模的样本行数: 57 / 原始 59
模型比较（Top 按调整R^2）:
AdjR2=0.660 | cond_no=24997.1 | n=57 | AIC=-29.7 | ln_value ~ ln_value_lag1 + ln_value_lag2 + ln_tariff + ln_tariff_lag1 + ln_tariff_lag2 + cov19 + C(Month) + C(Year)
AdjR2=0.642 | cond_no=23978.8 | n=57 | AIC=-29.2 | ln_value ~ ln_value_lag1 + ln_value_lag2 + ln_tariff + ln_tariff_lag1 + cov19 + C(Month)
AdjR2=0.626 | cond_no=14238.0 | n=57 | AIC=-28.0 | ln_value ~ ln_value_lag1 + ln_tariff + cov19 + C(Month)
AdjR2=0.619 | cond_no=29.0 | n=57 | AIC=-23.7 | ln_value ~ ln_value_lag1_z + ln_value_lag2_z + ln_tariff_z + ln_tariff_lag1_z + cov19 + C(Month) + C(Year)
AdjR2=0.598 | cond_no=14483.2 | n=57 | AIC=-21.4 | ln_value ~ ln_value_lag1 + ln_tariff + cov19 + C(Month) + C(Year)
AdjR2=0.590 | cond_no=25.8 | n=57 | AIC=-19.9 | ln_value ~ ln_value_lag1_z + ln_tariff_z * cov19 + C(Month) + C(Year)
AdjR2=0.483 | cond_no=12605.3 | n=57 | AIC=-18.2 | ln_value ~ ln_value_lag1 + ln_tariff + cov19
AdjR2=0.360 | cond_no=704.2 | n=57 | AIC=-8.5 | d_ln_value ~

In [36]:
# prediction based on final_fit
print(final_fit.summary())

                            OLS Regression Results                            
Dep. Variable:               ln_value   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     6.181
Date:                Sun, 23 Nov 2025   Prob (F-statistic):           1.29e-06
Time:                        20:42:44   Log-Likelihood:                 36.869
No. Observations:                  57   AIC:                            -29.74
Df Residuals:                      35   BIC:                             15.21
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          17.9169      3.178     

In [51]:
# predict based on below fact
#                             OLS Regression Results                            
# ==============================================================================
# Dep. Variable:               ln_value   R-squared:                       0.788
# Model:                            OLS   Adj. R-squared:                  0.660
# Method:                 Least Squares   F-statistic:                     6.181
# Date:                Sun, 23 Nov 2025   Prob (F-statistic):           1.29e-06
# Time:                        20:42:44   Log-Likelihood:                 36.869
# No. Observations:                  57   AIC:                            -29.74
# Df Residuals:                      35   BIC:                             15.21
# Df Model:                          21                                         
# Covariance Type:            nonrobust                                         
# ===================================================================================
#                       coef    std err          t      P>|t|      [0.025      0.975]
# -----------------------------------------------------------------------------------
# Intercept          17.9169      3.178      5.639      0.000      11.466      24.368
# C(Month)[T.2]      -0.0544      0.133     -0.409      0.685      -0.324       0.215
# C(Month)[T.3]       0.2173      0.139      1.565      0.127      -0.065       0.499
# C(Month)[T.4]      -0.0697      0.111     -0.630      0.532      -0.294       0.155
# C(Month)[T.5]      -0.0737      0.119     -0.617      0.541      -0.316       0.169
# C(Month)[T.6]      -0.0534      0.126     -0.425      0.674      -0.308       0.202
# C(Month)[T.7]       0.2185      0.126      1.732      0.092      -0.038       0.475
# C(Month)[T.8]       0.0273      0.116      0.236      0.815      -0.208       0.262
# C(Month)[T.9]      -0.0869      0.119     -0.731      0.470      -0.328       0.155
# C(Month)[T.10]      0.1529      0.127      1.204      0.237      -0.105       0.411
# C(Month)[T.11]     -0.0477      0.118     -0.403      0.689      -0.288       0.193
# C(Month)[T.12]      0.1166      0.125      0.936      0.356      -0.136       0.369
# C(Year)[T.2021]    -0.0243      0.081     -0.301      0.765      -0.188       0.139
# C(Year)[T.2022]    -0.0400      0.081     -0.493      0.625      -0.205       0.125
# C(Year)[T.2023]    -0.3414      0.221     -1.548      0.131      -0.789       0.106
# C(Year)[T.2024]    -0.3259      0.221     -1.473      0.150      -0.775       0.123
# ln_value_lag1       0.5830      0.148      3.950      0.000       0.283       0.883
# ln_value_lag2      -0.3046      0.137     -2.216      0.033      -0.584      -0.026
# ln_tariff         -38.6480     14.938     -2.587      0.014     -68.975      -8.321
# ln_tariff_lag1    -11.4277     15.641     -0.731      0.470     -43.180      20.324
# ln_tariff_lag2    -37.8278     16.405     -2.306      0.027     -71.132      -4.524
# cov19              -0.4011      0.196     -2.048      0.048      -0.799      -0.003
# ==============================================================================
# Omnibus:                        5.555   Durbin-Watson:                   2.214
# Prob(Omnibus):                  0.062   Jarque-Bera (JB):                4.533
# Skew:                          -0.626   Prob(JB):                        0.104
# Kurtosis:                       3.583   Cond. No.                     2.50e+04
# ==============================================================================

# get final parameters
params = final_fit.params
alpha = params['Intercept']
rho1 = params['ln_value_lag1']
rho2 = params['ln_value_lag2']
beta0 = params['ln_tariff']
beta1 = params['ln_tariff_lag1']
beta2 = params['ln_tariff_lag2']
gamma = params['cov19']
print("\n参数：", alpha, rho1, rho2, beta0, beta1, beta2, gamma)
# =======================
# 6. 预测 2025年1月的 ln(value)
# =======================
# 取 2024年12月 和 2024年11月 作为滞后项
last = df[df["Year"]==2024].sort_values("Month").iloc[-1]
ln_value_last1 = last["ln_value"]
last2 = df[(df["Year"]==2024) & (df["Month"]==11)].iloc[0]
ln_value_last2 = last2["ln_value"]
# 取 2025.01 的关税（若没有你需要自己填写或假设不变）
tariff_2025_01 = last["tariff"]   # 假设不变
ln_tariff_2025_01 = np.log(1 + tariff_2025_01)
# 计算 ln(value_2025_01)
ln_val_2025_01 = (alpha + rho1 * ln_value_last1 + rho2 * ln_value_last2 +
                  beta0 * ln_tariff_2025_01 +
                  beta1 * np.log(1 + last2["tariff"]) +  # 假设11月关税同12月
                  beta2 * np.log(1 + df[(df["Year"]==2024) & (df["Month"]==10)].iloc[0]["tariff"]) +  # 假设10月关税同11月
                  gamma * last["cov19"])  # 假设cov19同12月
# 转回 trade total 值
val_2025_01 = np.exp(ln_val_2025_01)
print("\n预测 2025年1月 日本总出口额（value）为：", val_2025_01)


参数： 17.916901757257637 0.5829923785735288 -0.3046091012479278 -38.647960937962324 -11.427708063286573 -37.827785336497755 -0.40107702652783195

预测 2025年1月 日本总出口额（value）为： 5002256014.788939


In [54]:
from scipy.optimize import linprog
import numpy as np

# ================= 改进的生产优化线性规划 =================
# 问题：之前直接使用 Q_jap_2025 = val_2025_01 / 0.190 并令 Q_2025 = Q_jap_2025
# 再用每个产量 ±5% 作为上下界，若目标总产量不落在 sum(lower_bounds) 与 sum(upper_bounds) 之间，会导致不可行 => res.x 为 None。
# 改进点：
# 1. 明确总产量的来源：用 2024 各产地量求出总量，再按增长率假设得到 2025.01 的目标。
# 2. 在求解前进行可行性检查：若目标不在范围内，自动扩大灵活度 flex 或提示重新设定。
# 3. 多次尝试放宽上/下界，仍失败则给出诊断信息。
# 4. 支持传入各地成本与最小/最大调整百分比。

# ---------------- 输入（请根据真实数据调整） ----------------
# 2024 各地产量（示例）
x1_2024 = 3678000802.00  # 日本
x2_2024 = 4451862982.00  # 墨西哥
x3_2024 = int((3678000802.00/0.2) - 3678000802.00 - 4451862982.00)  # 其他 (示例公式, 建议替换)

base_outputs = np.array([x1_2024, x2_2024, x3_2024], dtype=float)
# 成本（单位成本或系数）：越低越倾向分配更多产量
costs = np.array([115, 77, 85], dtype=float)

# 日本一月预测出口额（来自上文 val_2025_01）
# 这里 val_2025_01 已在前面计算；假设其占总量比例 share_jap (如 0.190)
share_jap = 0.190
Q_jap_2025 = val_2025_01 / share_jap

# 可以设置一个总增长率（例如总量同比增长 2%）
assumed_growth_rate = 0.02  # 2% 增长，可调
Q_base_total_2024 = base_outputs.sum()
Q_2025_target = Q_base_total_2024 * (1 + assumed_growth_rate)

# 若希望以日本预测反推总量，可选择：Q_2025_target = Q_jap_2025
# 请注释/取消注释其中一行：
# Q_2025_target = Q_jap_2025

print(f"2024 总产量 = {Q_base_total_2024:,.0f}")
print(f"预测 2025.01 总产量目标 = {Q_2025_target:,.0f}")
print(f"日本占比假设 share_jap={share_jap} => 总量反推(若使用) = {Q_jap_2025:,.0f}")

# ---------------- 线性规划函数 ----------------

def solve_production(target_total, last_outputs, costs, flex=0.05, max_expand=0.30, step=0.05):
    """最小化成本 subject to sum(x)=target_total, 每个产量在 [ (1-flex)*x_prev , (1+flex)*x_prev ].
    若不可行自动增加 flex，直到 max_expand 或成功。"""
    prev = last_outputs.astype(float)
    current_flex = flex
    attempt = 0
    while current_flex <= max_expand:
        lo_bounds = prev * (1 - current_flex)
        hi_bounds = prev * (1 + current_flex)
        total_lo = lo_bounds.sum()
        total_hi = hi_bounds.sum()
        print(f"尝试 flex={current_flex:.2f} => 可行区间[{total_lo:,.0f}, {total_hi:,.0f}]")
        if not (total_lo <= target_total <= total_hi):
            # 不可行，需要放宽
            current_flex += step
            attempt += 1
            continue
        # 构造 LP
        bounds = list(zip(lo_bounds, hi_bounds))
        A_eq = np.array([[1, 1, 1]])
        b_eq = np.array([target_total])
        res = linprog(costs, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
        if res.success:
            return {"success": True, "flex": current_flex, "result": res}
        else:
            print("LP 求解失败，放宽区间继续尝试……")
            current_flex += step
            attempt += 1
    return {"success": False, "flex": current_flex, "message": "达到最大放宽仍未找到可行解"}

# ---------------- 执行求解 ----------------
solution = solve_production(Q_2025_target, base_outputs, costs, flex=0.05, max_expand=0.30, step=0.05)

if solution['success']:
    res = solution['result']
    print("\n优化成功：")
    print(f"使用 flex={solution['flex']:.2f}")
    x1_2025, x2_2025, x3_2025 = res.x
    print(f"x1_2025 = {x1_2025:,.0f}")
    print(f"x2_2025 = {x2_2025:,.0f}")
    print(f"x3_2025 = {x3_2025:,.0f}")
    print(f"总成本 = {res.fun:,.2f}")
    print(f"检查总和 = {x1_2025 + x2_2025 + x3_2025:,.0f}")
else:
    print("\n优化未成功：", solution['message'])
    print("建议：1) 调整增长率或 share_jap；2) 增大 max_expand；3) 放宽某个产地单独约束；4) 使用不等式约束 (>=) 改造模型。")

# ---------------- 可选：按成本排序的产能分配解释 ----------------
if solution.get('success'):
    alloc = np.array([x1_2025, x2_2025, x3_2025])
    share = alloc / alloc.sum()
    explain = sorted(zip(['Japan','Mexico','Other'], alloc, share, costs), key=lambda z: z[3])
    print("\n按单位成本排序的产量与份额：")
    for name, qty, sh, c in explain:
        print(f"{name:<6} 成本={c:>6.1f} 产量={qty:>12,.0f} 份额={sh*100:5.2f}%")

# ================= 结束 =================


2024 总产量 = 18,390,004,010
预测 2025.01 总产量目标 = 18,757,804,090
日本占比假设 share_jap=0.19 => 总量反推(若使用) = 26,327,663,236
尝试 flex=0.05 => 可行区间[17,470,503,810, 19,309,504,210]

优化成功：
使用 flex=0.05
x1_2025 = 3,494,100,762
x2_2025 = 4,674,456,131
x3_2025 = 10,589,247,197
总成本 = 1,661,840,721,475.20
检查总和 = 18,757,804,090

按单位成本排序的产量与份额：
Mexico 成本=  77.0 产量=4,674,456,131 份额=24.92%
Other  成本=  85.0 产量=10,589,247,197 份额=56.45%
Japan  成本= 115.0 产量=3,494,100,762 份额=18.63%


In [49]:

# #---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# Cell In[47], line 34
#      31 print(x_2024[2])
#      33 print("优化结果 2025.01:")
# ---> 34 print(f"x1_2025 = {res.x[0]}")
#      35 print(f"x2_2025 = {res.x[1]}")
#      36 print(f"x3_2025 = {res.x[2]}")

# TypeError: 'NoneType' object is not subscriptable

#fix error 
if res.success:
    print("优化结果 2025.01:")
    print(f"x1_2025 = {res.x[0]}")
    print(f"x2_2025 = {res.x[1]}")
    print(f"x3_2025 = {res.x[2]}")
    print("\n总成本 =", res.fun)
      